![Redis](https://redis.io/wp-content/uploads/2024/04/Logotype.svg?auto=webp&quality=85,75&width=120)

# Module 4: Memory Systems

**⏱️ Estimated Time:** 75-90 minutes

## 🎯 Learning Objectives

By the end of this module, you will:

1. **Understand** why memory is essential for context engineering
2. **Implement** working memory for conversation continuity
3. **Use** long-term memory for persistent user knowledge
4. **Integrate** memory with your Module 2 RAG system
5. **Build** a complete memory-enhanced course advisor
6. **Combine** all four context types in a unified system

---

## 🔗 Recap

### **Module 1: The Four Context Types**

Recall the four context types from Module 1:

1. **System Context** (Static) - Role, instructions, guidelines
2. **User Context** (Dynamic, User-Specific) - Profile, preferences, goals
3. **Conversation Context** (Dynamic, Session-Specific) - **← Memory enables this!**
4. **Retrieved Context** (Dynamic, Query-Specific) - RAG results

### **Module 2: Stateless RAG**

Your Module 2 RAG system was **stateless**:

```python
async def rag_query(query, student_profile):
    # 1. Search courses (Retrieved Context)
    courses = await course_manager.search_courses(query)

    # 2. Assemble context (System + User + Retrieved)
    context = assemble_context(system_prompt, student_profile, courses)

    # 3. Generate response
    response = llm.invoke(context)

    # ❌ No conversation history stored
    # ❌ Each query is independent
    # ❌ Can't reference previous messages
```

**The Problem:** Every query starts from scratch. No conversation continuity.

---

## 🚨 Why Agents Need Memory: The Grounding Problem

Before diving into implementation, let's understand the fundamental problem that memory solves.

**Grounding** means understanding what users are referring to. Natural conversation is full of references:

### **Without Memory:**

```
User: "Tell me about CS401"
Agent: "CS401 is Machine Learning. It covers supervised learning..."

User: "What are its prerequisites?"
Agent: ❌ "What does 'it' refer to? Please specify which course."

User: "The course we just discussed!"
Agent: ❌ "I don't have access to previous messages. Which course?"
```

**This is a terrible user experience.**

### Types of References That Need Grounding

**Pronouns:**
- "it", "that course", "those", "this one"
- "he", "she", "they" (referring to people)

**Descriptions:**
- "the easy one", "the online course"
- "my advisor", "that professor"

**Implicit context:**
- "Can I take it?" → Take what?
- "When does it start?" → What starts?

**Temporal references:**
- "you mentioned", "earlier", "last time"

### **With Memory:**

```
User: "Tell me about CS401"
Agent: "CS401 is Machine Learning. It covers..."
[Stores: User asked about CS401]

User: "What are its prerequisites?"
Agent: [Checks memory: "its" = CS401]
Agent: ✅ "CS401 requires CS201 and MATH301"

User: "Can I take it?"
Agent: [Checks memory: "it" = CS401, checks student transcript]
Agent: ✅ "You've completed CS201 but still need MATH301"
```

**Now the conversation flows naturally!**

---

## 🧠 Two Types of Memory

### **1. Working Memory (Session-Scoped)**

 - **What:** Conversation messages from the current session
 - **Purpose:** Reference resolution, conversation continuity
 - **Lifetime:** Persists for the session
 - **Storage:** Conversation remains accessible when you return to the same session

**Example:**
```
Session: session_123
Messages:
  1. User: "Tell me about CS401"
  2. Agent: "CS401 is Machine Learning..."
  3. User: "What are its prerequisites?"
  4. Agent: "CS401 requires CS201 and MATH301"
```

**Key Point:** Just like ChatGPT or Claude, when you return to a conversation, the working memory is still there. The conversation doesn't disappear!

### **2. Long-term Memory (Cross-Session)**

 - **What:** Persistent knowledge (user preferences, domain facts, business rules)
 - **Purpose:** Personalization AND consistent application behavior across sessions
 - **Lifetime:** Permanent (until explicitly deleted)
 - **Scope:** Can be user-specific OR application-wide

**Examples:**

**User-Scoped (Personalization):**
```
User: student_sarah
  - "Prefers online courses over in-person"
  - "Major: Computer Science, focus on AI/ML"
  - "Goal: Graduate Spring 2026"
  - "Completed: CS101, CS201, MATH301"
```

**Application-Scoped (Domain Knowledge):**
```
Domain: course_requirements
  - "CS401 requires CS201 as prerequisite"
  - "Maximum course load is 18 credits per semester"
  - "Registration opens 2 weeks before semester start"
  - "Lab courses require campus attendance"
```

### **Comparison: Working vs. Long-term Memory**

| Working Memory | Long-term Memory |
|----------------|------------------|
| **Session-scoped** | **User-scoped OR Application-scoped** |
| Current conversation | Important facts, rules, knowledge |
| Persists for session | Persists across sessions |
| Full message history | Extracted knowledge (user + domain) |
| Loaded/saved each turn | Searched when needed |
| **Challenge:** Context window limits | **Challenge:** Storage growth |

---

## 📦 Setup and Environment

Let's set up our environment with the necessary dependencies and connections. We'll build on Module 2's RAG foundation and add memory capabilities.

### ⚠️ Prerequisites

**Before running this notebook, make sure you have:**

1. **Docker Desktop running** - Required for Redis and Agent Memory Server

2. **Environment variables** - Create a `.env` file in the `reference-agent` directory:
   ```bash
   # Copy the example file
   cd ../reference-agent
   cp .env.example .env

   # Edit .env and add your OpenAI API key
   # OPENAI_API_KEY=your_actual_openai_api_key_here
   ```

3. **Run the setup script** - This will automatically start Redis and Agent Memory Server:
   ```bash
   cd ../reference-agent
   python setup_agent_memory_server.py
   ```

**Note:** The setup script will:
- ✅ Check if Docker is running
- ✅ Start Redis if not running (port 6379)
- ✅ Start Agent Memory Server if not running (port 8088)
- ✅ Verify Redis connection is working
- ✅ Handle any configuration issues automatically

If the Memory Server is not available, the notebook will skip memory-related demos but will still run.


---


### Automated Setup Check

Let's run the setup script to ensure all services are running properly.


In [1]:
# Run the setup script to ensure Redis and Agent Memory Server are running
import subprocess
import sys
from pathlib import Path

# Path to setup script
setup_script = Path("../reference-agent/setup_agent_memory_server.py")

if setup_script.exists():
    print("Running automated setup check...\n")
    result = subprocess.run(
        [sys.executable, str(setup_script)], capture_output=True, text=True
    )
    print(result.stdout)
    if result.returncode != 0:
        print("⚠️  Setup check failed. Please review the output above.")
        print(result.stderr)
    else:
        print("\n✅ All services are ready!")
else:
    print("⚠️  Setup script not found. Please ensure services are running manually.")

⚠️  Setup script not found. Please ensure services are running manually.


---


### Install Dependencies

If you haven't already installed the reference-agent package, uncomment and run the following:


In [2]:
# Uncomment to install reference-agent package
# %pip install -q -e ../reference-agent

# Uncomment to install agent-memory-client
# %pip install -q agent-memory-client

### Load Environment Variables

We'll load environment variables from the `.env` file in the `reference-agent` directory.

**Required variables:**
- `OPENAI_API_KEY` - Your OpenAI API key
- `REDIS_URL` - Redis connection URL (default: redis://localhost:6379)
- `AGENT_MEMORY_URL` - Agent Memory Server URL (default: http://localhost:8088)

If you haven't created the `.env` file yet, copy `.env.example` and add your OpenAI API key.


In [3]:
import os
import sys
from pathlib import Path

from dotenv import load_dotenv

# Handle both running from workshop/ directory and from project root
if Path.cwd().name == "workshop":
    project_root = Path.cwd().parent
else:
    project_root = Path.cwd()

if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Load environment variables from project root first, then reference-agent
env_path = project_root / ".env"
load_dotenv(dotenv_path=env_path)

# Also try reference-agent .env for memory-specific settings
ref_agent_env = project_root / "reference-agent" / ".env"
if ref_agent_env.exists():
    load_dotenv(dotenv_path=ref_agent_env, override=False)

# Verify required environment variables
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
REDIS_URL = os.getenv("REDIS_URL", "redis://localhost:6379")
AGENT_MEMORY_URL = os.getenv("AGENT_MEMORY_URL", "http://localhost:8088")

if not OPENAI_API_KEY:
    print(
        f"""❌ OPENAI_API_KEY not found!

    Please create a .env file at: {env_path.absolute()}

    With the following content:
    OPENAI_API_KEY=your_openai_api_key
    REDIS_URL=redis://localhost:6379
    AGENT_MEMORY_URL=http://localhost:8088
    """
    )
else:
    print(f"""✅ Environment variables loaded
   REDIS_URL: {REDIS_URL}
   AGENT_MEMORY_URL: {AGENT_MEMORY_URL}""")

✅ Environment variables loaded
   REDIS_URL: redis://localhost:6379
   AGENT_MEMORY_URL: http://localhost:8088


### Import Core Libraries

We'll import standard Python libraries and async support for our memory operations.


In [4]:
import asyncio
import json
import uuid
from datetime import datetime
from typing import Optional

import nest_asyncio

# Enable nested event loops (required for Jupyter)
nest_asyncio.apply()

print("✅ Core libraries imported")

✅ Core libraries imported


### Import Module 2 Components

We're building on Module 2's RAG foundation, so we'll reuse the same components:
- `redis_config` - Redis connection and configuration
- `HierarchicalCourseManager` - Two-tier course search (summaries + details)
- `HierarchicalContextAssembler` - Progressive disclosure context assembly
- `StudentProfile` and other models - Data structures


In [5]:
from redis_context_course.hierarchical_context import HierarchicalContextAssembler
from redis_context_course.hierarchical_manager import HierarchicalCourseManager
from redis_context_course.models import (
    Course,
    CourseFormat,
    DifficultyLevel,
    Semester,
    StudentProfile,
)

# Import Redis configuration from reference-agent
from redis_context_course.redis_config import redis_config

print("""✅ Module 2 components imported
   HierarchicalCourseManager: Available
   HierarchicalContextAssembler: Available
   Redis Config: Available
   Models: Course, StudentProfile, etc.""")

✅ Module 2 components imported
   HierarchicalCourseManager: Available
   HierarchicalContextAssembler: Available
   Redis Config: Available
   Models: Course, StudentProfile, etc.


### Import LangChain Components

We'll use LangChain for LLM interaction and message handling.


In [6]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI

print("""✅ LangChain components imported
   ChatOpenAI: Available
   Message types: HumanMessage, SystemMessage, AIMessage""")

✅ LangChain components imported
   ChatOpenAI: Available
   Message types: HumanMessage, SystemMessage, AIMessage


### Import Agent Memory Server Client

The Agent Memory Server provides production-ready memory management. If it's not available, we'll note that and continue with limited functionality.


In [7]:
# Import Agent Memory Server client
try:
    from agent_memory_client import MemoryAPIClient, MemoryClientConfig
    from agent_memory_client.models import (
        ClientMemoryRecord,
        MemoryMessage,
        WorkingMemory,
    )

    MEMORY_SERVER_AVAILABLE = True
    print("""✅ Agent Memory Server client available
   MemoryAPIClient: Ready
   Memory models: WorkingMemory, MemoryMessage, ClientMemoryRecord""")
except ImportError:
    MEMORY_SERVER_AVAILABLE = False
    print("""⚠️  Agent Memory Server not available
   Install with: pip install agent-memory-client
   Start server: See reference-agent/README.md
   Note: Some demos will be skipped""")

✅ Agent Memory Server client available
   MemoryAPIClient: Ready
   Memory models: WorkingMemory, MemoryMessage, ClientMemoryRecord


### What We Just Did

We've successfully set up our environment with all the necessary components:

**Imported:**
- ✅ Module 2 RAG components (`HierarchicalCourseManager`, `HierarchicalContextAssembler`, `redis_config`, models)
- ✅ LangChain for LLM interaction
- ✅ Agent Memory Server client (if available)

**Why This Matters:**
- Building on Module 2's foundation (not starting from scratch)
- Using progressive disclosure pattern (summaries → details)
- Agent Memory Server provides scalable, persistent memory
- Same Redis University domain for consistency

---

## 🔧 Initialize Components

Now let's initialize the components we'll use throughout this notebook.


### Initialize Redis Connection

First, let's connect to Redis using the same configuration from Module 2.


In [8]:
# Initialize Redis connection (redis_config.redis_client is a property)
redis_client = redis_config.redis_client

print(f"""✅ Redis connection established
   URL: {REDIS_URL}
   Ready for vector operations""")

✅ Redis connection established
   URL: redis://localhost:6379
   Ready for vector operations


### Initialize Hierarchical Course Manager

The `HierarchicalCourseManager` provides two-tier retrieval:
- **Tier 1:** Course summaries (lightweight, for search)
- **Tier 2:** Full course details (on-demand)

This is the same progressive disclosure pattern from Module 2.


In [9]:
# Initialize Hierarchical Course Manager
hierarchical_manager = HierarchicalCourseManager(redis_client=redis_client)
context_assembler = HierarchicalContextAssembler()

print("""✅ Hierarchical Course Manager initialized
   Two-tier retrieval: summaries → details
   Progressive disclosure pattern ready""")

✅ Hierarchical Course Manager initialized
   Two-tier retrieval: summaries → details
   Progressive disclosure pattern ready


### Initialize LLM

We'll use GPT-4o with temperature=0.0 for consistent, deterministic responses.


In [10]:
# Initialize LLM
llm = ChatOpenAI(model="gpt-4o", temperature=0.0)

print("✅ LLM initialized (GPT-4o)")

✅ LLM initialized (GPT-4o)


### Initialize Memory Client

If the Agent Memory Server is available, we'll initialize the memory client. This client handles both working memory (conversation history) and long-term memory (persistent facts).


In [11]:
# Initialize Memory Client
if MEMORY_SERVER_AVAILABLE:
    config = MemoryClientConfig(
        base_url=AGENT_MEMORY_URL, default_namespace="redis_university"
    )
    memory_client = MemoryAPIClient(config=config)
    print(f"""✅ Memory Client initialized
   Base URL: {config.base_url}
   Namespace: {config.default_namespace}
   Ready for working memory and long-term memory operations""")
else:
    memory_client = None
    print("""⚠️  Memory Server not available
   Running with limited functionality
   Some demos will be skipped""")

✅ Memory Client initialized
   Base URL: http://localhost:8088
   Namespace: redis_university
   Ready for working memory and long-term memory operations


### Create Sample Student Profile

We'll create a sample student profile to use throughout our demos. This follows the same pattern from Module 2.


In [12]:
# Create sample student profile
sarah = StudentProfile(
    name="Sarah Chen",
    email="sarah.chen@university.edu",
    major="Computer Science",
    year=2,
    interests=["machine learning", "data science", "algorithms"],
    completed_courses=["CS101", "CS201"],
    current_courses=["MATH301"],
    preferred_format=CourseFormat.ONLINE,
    preferred_difficulty=DifficultyLevel.INTERMEDIATE,
)

print(f"""✅ Student profile created
   Name: {sarah.name}
   Major: {sarah.major}
   Year: {sarah.year}
   Interests: {', '.join(sarah.interests)}
   Completed: {', '.join(sarah.completed_courses)}
   Preferred Format: {sarah.preferred_format.value}""")

✅ Student profile created
   Name: Sarah Chen
   Major: Computer Science
   Year: 2
   Interests: machine learning, data science, algorithms
   Completed: CS101, CS201
   Preferred Format: online


In [13]:
print(f"""🎯 INITIALIZATION SUMMARY

✅ Redis Connection: Ready
✅ Hierarchical Course Manager: Ready (two-tier retrieval)
✅ Context Assembler: Ready (progressive disclosure)
✅ LLM (GPT-4o): Ready
{'✅' if MEMORY_SERVER_AVAILABLE else '⚠️ '} Memory Client: {'Ready' if MEMORY_SERVER_AVAILABLE else 'Not Available'}
✅ Student Profile: {sarah.name}""")

🎯 INITIALIZATION SUMMARY

✅ Redis Connection: Ready
✅ Hierarchical Course Manager: Ready (two-tier retrieval)
✅ Context Assembler: Ready (progressive disclosure)
✅ LLM (GPT-4o): Ready
✅ Memory Client: Ready
✅ Student Profile: Sarah Chen


### Initialization Done

📋 **What We're Building On:**
- Module 2's RAG foundation (`HierarchicalCourseManager`, `redis_config`)
- Same `StudentProfile` model
- Same Redis configuration
- Progressive disclosure pattern (summaries → details)

✨ **What We're Adding:**
- Memory Client for conversation history
- Working Memory for session context
- Long-term Memory for persistent knowledge


---

## 📚 Part 1: Working Memory Fundamentals

### **What is Working Memory?**

Working memory stores **conversation messages** for the current session. It enables:

- ✅ **Reference resolution** - "it", "that course", "the one you mentioned"
- ✅ **Context continuity** - Each message builds on previous messages
- ✅ **Natural conversations** - Users don't repeat themselves

### **How It Works:**

```
Turn 1: Load working memory (empty) → Process query → Save messages
Turn 2: Load working memory (1 exchange) → Process query → Save messages
Turn 3: Load working memory (2 exchanges) → Process query → Save messages
```

Each turn has access to all previous messages in the session.

---

## 🧪 Hands-On: Working Memory in Action

Let's simulate a multi-turn conversation with working memory. We'll break this down step-by-step to see how working memory enables natural conversation flow.


### Setup: Create Session and Student IDs

Now that we have our components initialized, let's create session and student identifiers for our working memory demo.


In [14]:
# Setup for working memory demo
student_id = sarah.email.split("@")[0]  # "sarah.chen"
session_id = f"session_{student_id}_demo"

print(f"""🎯 Working Memory Demo Setup
   Student ID: {student_id}
   Session ID: {session_id}
   Ready to demonstrate multi-turn conversation""")

🎯 Working Memory Demo Setup
   Student ID: sarah.chen
   Session ID: session_sarah.chen_demo
   Ready to demonstrate multi-turn conversation


### Turn 1: Initial Query

Let's start with a simple query about a course. This is the first turn, so working memory will be empty.

We'll break this down into clear steps:
1. Load working memory (will be empty on first turn)
2. Search for courses using hierarchical retrieval
3. Generate a response
4. Save the conversation to working memory


#### Step 1: Set up the user query


In [15]:
print("=" * 80)
print("📍 TURN 1: User asks about a course")
print("=" * 80)

# Define the user's query
turn1_query = "Tell me about machine learning courses"
print(f"\n👤 User: {turn1_query}")

📍 TURN 1: User asks about a course

👤 User: Tell me about machine learning courses


#### Step 2: Load working memory

On the first turn, working memory will be empty since this is a new session.


In [16]:
if MEMORY_SERVER_AVAILABLE:
    # Load working memory (empty for first turn)
    _, turn1_working_memory = await memory_client.get_or_create_working_memory(
        session_id=session_id, user_id=student_id, model_name="gpt-4o"
    )

    print(f"""📊 Working Memory Status:
   Messages in memory: {len(turn1_working_memory.messages)}
   Status: {'Empty (first turn)' if len(turn1_working_memory.messages) == 0 else 'Has history'}""")

19:17:02 httpx INFO   HTTP Request: GET http://localhost:8088/v1/working-memory/session_sarah.chen_demo?user_id=sarah.chen&namespace=redis_university&model_name=gpt-4o "HTTP/1.1 200 OK"


📊 Working Memory Status:
   Messages in memory: 6
   Status: Has history


#### Step 3: Search for courses using hierarchical retrieval

Use the hierarchical manager to search for courses. This uses the progressive disclosure pattern:
- First, get summaries (lightweight)
- Then, fetch details for top results (on-demand)


In [17]:
print("\n🔍 Searching for courses using hierarchical retrieval...")

# Use hierarchical search (summaries + details)
turn1_summaries, turn1_details = await hierarchical_manager.hierarchical_search(
    query=turn1_query,
    summary_limit=3,
    detail_limit=2
)

print(f"""   Found {len(turn1_summaries)} summaries, fetched {len(turn1_details)} details
   Progressive disclosure: summaries first, details on-demand""")

# Show what we found
if turn1_summaries:
    print("\n   📋 Course Summaries:")
    for i, summary in enumerate(turn1_summaries[:3], 1):
        print(f"      {i}. {summary.course_code}: {summary.title}")


🔍 Searching for courses using hierarchical retrieval...
19:17:02 redis_context_course.hierarchical_manager INFO   Hierarchical search: 'Tell me about machine learning courses' (summaries=3, details=2)


19:17:03 httpx INFO   HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


19:17:03 redisvl.index.index INFO   Index already exists, not overwriting.


19:17:03 redis_context_course.hierarchical_manager INFO   Created summary index: course_summaries


19:17:03 redis_context_course.hierarchical_manager INFO   Found 0 course summaries for query: Tell me about machine learning courses


19:17:03 redis_context_course.hierarchical_manager INFO   Fetched 0 course details


19:17:03 redis_context_course.hierarchical_manager INFO   Hierarchical search complete: 0 summaries, 0 details


   Found 0 summaries, fetched 0 details
   Progressive disclosure: summaries first, details on-demand


#### Step 4: Assemble context and generate response

Use the context assembler to build context with progressive disclosure, then generate a response.


In [18]:
# Assemble context using progressive disclosure
turn1_context = context_assembler.assemble_hierarchical_context(
    summaries=turn1_summaries,
    details=turn1_details,
    query=turn1_query
)

print(f"   📝 Context assembled ({len(turn1_context)} characters)")

# Build messages for LLM
turn1_messages = [
    SystemMessage(
        content="You are a helpful course advisor. Answer questions about courses based on the provided information. Be concise but informative."
    ),
    HumanMessage(content=f"{turn1_context}\n\nUser question: {turn1_query}"),
]

# Generate response using LLM
print("\n💭 Generating response using LLM...")
turn1_response = llm.invoke(turn1_messages).content

print(f"\n🤖 Agent: {turn1_response}")

   📝 Context assembled (123 characters)

💭 Generating response using LLM...


19:17:05 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



🤖 Agent: It seems there are no specific machine learning courses listed in the search results you provided. However, generally speaking, machine learning courses typically cover topics such as supervised and unsupervised learning, neural networks, deep learning, and data preprocessing. They often include practical components where students work on projects using programming languages like Python and tools such as TensorFlow or PyTorch. If you're interested in machine learning, you might want to explore online platforms like Coursera, edX, or Udacity, which offer a variety of courses ranging from beginner to advanced levels.


#### Step 5: Save to working memory

Add both the user query and assistant response to working memory for future turns.


In [19]:
if MEMORY_SERVER_AVAILABLE:
    # Add messages to working memory
    turn1_working_memory.messages.extend(
        [
            MemoryMessage(role="user", content=turn1_query),
            MemoryMessage(role="assistant", content=turn1_response),
        ]
    )

    # Save to Memory Server
    await memory_client.put_working_memory(
        session_id=session_id,
        memory=turn1_working_memory,
        user_id=student_id,
        model_name="gpt-4o",
    )

    print(f"""
✅ Saved to working memory
   Messages now in memory: {len(turn1_working_memory.messages)}""")

19:17:05 httpx INFO   HTTP Request: PUT http://localhost:8088/v1/working-memory/session_sarah.chen_demo?user_id=sarah.chen&model_name=gpt-4o "HTTP/1.1 200 OK"



✅ Saved to working memory
   Messages now in memory: 8


### What Just Happened in Turn 1?

**Initial State:**
- Working memory was empty (first turn)
- No conversation history available

**Actions (RAG Pattern with Progressive Disclosure):**
1. **Retrieve:** Used hierarchical search (summaries → details)
2. **Augment:** Assembled context with progressive disclosure
3. **Generate:** LLM created a natural language response
4. **Save:** Stored conversation in working memory

**Result:**
- Working memory now contains 2 messages (1 user, 1 assistant)
- This history will be available for the next turn

**Key Insight:** We used the same hierarchical retrieval pattern from Module 2, now combined with memory!

---


### Turn 2: Follow-up with Pronoun Reference

Now let's ask a follow-up question using "it" - a pronoun that requires context from Turn 1.


#### Step 1: Set up the query


In [20]:
if MEMORY_SERVER_AVAILABLE:
    print("\n" + "=" * 80)
    print("📍 TURN 2: User uses pronoun reference ('it')")
    print("=" * 80)

    turn2_query = "What are the prerequisites for it?"
    print(f"\n👤 User: {turn2_query}")
    print("   Note: 'it' refers to a course from Turn 1")


📍 TURN 2: User uses pronoun reference ('it')

👤 User: What are the prerequisites for it?
   Note: 'it' refers to a course from Turn 1


#### Step 2: Load working memory

This time, working memory will contain the conversation from Turn 1.


In [21]:
if MEMORY_SERVER_AVAILABLE:
    # Load working memory (now has 1 exchange from Turn 1)
    _, turn2_working_memory = await memory_client.get_or_create_working_memory(
        session_id=session_id, user_id=student_id, model_name="gpt-4o"
    )

    print(f"""
📊 Working Memory Status:
   Messages in memory: {len(turn2_working_memory.messages)}
   Contains: Turn 1 conversation""")

19:17:05 httpx INFO   HTTP Request: GET http://localhost:8088/v1/working-memory/session_sarah.chen_demo?user_id=sarah.chen&namespace=redis_university&model_name=gpt-4o "HTTP/1.1 200 OK"



📊 Working Memory Status:
   Messages in memory: 8
   Contains: Turn 1 conversation


#### Step 3: Build context with conversation history

To resolve the pronoun "it", we need to include the conversation history in the LLM context.


In [22]:
if MEMORY_SERVER_AVAILABLE:
    print("\n🔧 Building context with conversation history...")

    # Start with system message
    turn2_messages = [
        SystemMessage(
            content="You are a helpful course advisor. Use conversation history to resolve references like 'it', 'that course', etc. Be concise but informative."
        )
    ]

    # Add conversation history from working memory
    for msg in turn2_working_memory.messages:
        if msg.role == "user":
            turn2_messages.append(HumanMessage(content=msg.content))
        elif msg.role == "assistant":
            turn2_messages.append(AIMessage(content=msg.content))

    # Add current query
    turn2_messages.append(HumanMessage(content=turn2_query))

    print(f"""   Total messages in context: {len(turn2_messages)}
   Includes: System prompt + Turn 1 history + current query""")


🔧 Building context with conversation history...
   Total messages in context: 10
   Includes: System prompt + Turn 1 history + current query


#### Step 4: Generate response using LLM

The LLM can now resolve "it" by looking at the conversation history.


In [23]:
if MEMORY_SERVER_AVAILABLE:
    print("\n💭 LLM resolving 'it' using conversation history...")
    turn2_response = llm.invoke(turn2_messages).content

    print(f"\n🤖 Agent: {turn2_response}")


💭 LLM resolving 'it' using conversation history...


19:17:08 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



🤖 Agent: The prerequisites for machine learning courses typically include:

1. **Mathematics**: A good understanding of linear algebra, calculus, probability, and statistics is essential, as these are foundational to many machine learning algorithms.

2. **Programming Skills**: Proficiency in a programming language, especially Python, is often required since it's widely used in machine learning for implementing algorithms and handling data.

3. **Basic Data Handling**: Familiarity with data manipulation and analysis, often using libraries like Pandas and NumPy, is important for working with datasets.

4. **Understanding of Algorithms**: A basic understanding of algorithms and data structures can be beneficial, as it helps in grasping how machine learning models work.

5. **Familiarity with Tools**: Some courses might expect you to have a basic understanding of machine learning frameworks and libraries, such as TensorFlow, Keras, or Scikit-learn.

These prerequisites ensure that you ha

#### Step 5: Save to working memory


In [24]:
if MEMORY_SERVER_AVAILABLE:
    # Add messages to working memory
    turn2_working_memory.messages.extend(
        [
            MemoryMessage(role="user", content=turn2_query),
            MemoryMessage(role="assistant", content=turn2_response),
        ]
    )

    # Save to Memory Server
    await memory_client.put_working_memory(
        session_id=session_id,
        memory=turn2_working_memory,
        user_id=student_id,
        model_name="gpt-4o",
    )

    print(f"""
✅ Saved to working memory
   Messages now in memory: {len(turn2_working_memory.messages)}""")

19:17:08 httpx INFO   HTTP Request: PUT http://localhost:8088/v1/working-memory/session_sarah.chen_demo?user_id=sarah.chen&model_name=gpt-4o "HTTP/1.1 200 OK"



✅ Saved to working memory
   Messages now in memory: 10


### What Just Happened in Turn 2?

**Initial State:**
- Working memory contained Turn 1 conversation (2 messages)
- User asked about "its prerequisites" - pronoun reference

**Actions:**
1. Loaded working memory with Turn 1 history
2. Built context including conversation history
3. LLM resolved "it" → the course from Turn 1
4. Generated response about prerequisites
5. Saved updated conversation to working memory

**Result:**
- Working memory now contains 4 messages (2 exchanges)
- LLM successfully resolved pronoun reference using conversation history
- Natural conversation flow maintained

**Key Insight:** Without working memory, the LLM wouldn't know what "it" refers to!

---


### Turn 3: Another Follow-up

Let's ask one more follow-up question to demonstrate continued conversation continuity.


In [25]:
if MEMORY_SERVER_AVAILABLE:
    print("\n" + "=" * 80)
    print("📍 TURN 3: User asks another follow-up")
    print("=" * 80)

    turn3_query = "Is it available online?"
    print(f"\n👤 User: {turn3_query}")
    print("   Note: 'it' still refers to the course from Turn 1")

    # Load working memory (now has 2 exchanges)
    _, turn3_working_memory = await memory_client.get_or_create_working_memory(
        session_id=session_id, user_id=student_id, model_name="gpt-4o"
    )

    print(f"""
📊 Working Memory Status:
   Messages in memory: {len(turn3_working_memory.messages)}
   Contains: Turns 1 and 2""")

    # Build context with full conversation history
    turn3_messages = [
        SystemMessage(
            content="You are a helpful course advisor. Use conversation history to resolve references."
        )
    ]

    for msg in turn3_working_memory.messages:
        if msg.role == "user":
            turn3_messages.append(HumanMessage(content=msg.content))
        elif msg.role == "assistant":
            turn3_messages.append(AIMessage(content=msg.content))

    turn3_messages.append(HumanMessage(content=turn3_query))

    print(f"   Total messages in context: {len(turn3_messages)}")

    # Generate response
    turn3_response = llm.invoke(turn3_messages).content

    print(f"\n🤖 Agent: {turn3_response}")

    # Save to working memory
    turn3_working_memory.messages.extend(
        [
            MemoryMessage(role="user", content=turn3_query),
            MemoryMessage(role="assistant", content=turn3_response),
        ]
    )

    await memory_client.put_working_memory(
        session_id=session_id,
        memory=turn3_working_memory,
        user_id=student_id,
        model_name="gpt-4o",
    )

    print(f"""
✅ Saved to working memory
   Messages now in memory: {len(turn3_working_memory.messages)}""")


📍 TURN 3: User asks another follow-up

👤 User: Is it available online?
   Note: 'it' still refers to the course from Turn 1
19:17:08 httpx INFO   HTTP Request: GET http://localhost:8088/v1/working-memory/session_sarah.chen_demo?user_id=sarah.chen&namespace=redis_university&model_name=gpt-4o "HTTP/1.1 200 OK"



📊 Working Memory Status:
   Messages in memory: 10
   Contains: Turns 1 and 2
   Total messages in context: 12


19:17:12 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



🤖 Agent: Yes, machine learning courses are widely available online. Many platforms offer a variety of courses that cater to different skill levels, from beginner to advanced. Here are some popular online platforms where you can find machine learning courses:

1. **Coursera**: Offers courses from universities like Stanford and institutions like Google, often including video lectures, assignments, and projects.

2. **edX**: Provides courses from universities such as MIT and Harvard, covering both introductory and advanced topics in machine learning.

3. **Udacity**: Known for its "Nanodegree" programs, which are more intensive and often include real-world projects and mentorship.

4. **Udemy**: Offers a wide variety of courses on machine learning, often at a lower cost, with options for beginners and more experienced learners.

5. **Khan Academy**: While not as comprehensive in machine learning specifically, it offers foundational courses in mathematics and programming.

6. **DataCamp**


✅ Saved to working memory
   Messages now in memory: 12


### 🎯 Working Memory Demo Summary

**📊 What Happened:**

| Turn | Query | Working Memory | Result |
|------|-------|----------------|--------|
| 1 | "Tell me about machine learning courses" | Empty (first turn) | Stored query + response |
| 2 | "What are the prerequisites for it?" | 1 exchange | LLM resolved 'it' using history |
| 3 | "Is it available online?" | 2 exchanges | Continued conversation flow |

**✅ Key Benefits:**
- Natural conversation flow
- Pronoun reference resolution
- No need to repeat context
- Seamless user experience

**❌ Without Working Memory:**
- "What are the prerequisites for it?" → "What is 'it'? Please specify."
- Each query is isolated
- User must repeat context every time

### Key Insight: Conversation Context Type

Working memory provides the **Conversation Context** - the third context type from Module 1:

1. **System Context** - Role and instructions (static)
2. **User Context** - Profile and preferences (dynamic, user-specific)
3. **Conversation Context** - Working memory (dynamic, session-specific) ← **We just demonstrated this!**
4. **Retrieved Context** - RAG results (dynamic, query-specific)

Without working memory, we only had 3 context types. Now we have all 4!


---

## 📚 Part 2: Long-term Memory for Context Engineering

### What is Long-term Memory?

Long-term memory enables AI agents to store **persistent knowledge** across sessions—including user preferences, domain facts, business rules, and system configuration. This is crucial for context engineering because it allows agents to:

- **Personalize** interactions by remembering user-specific preferences and history
- **Apply domain knowledge** consistently (prerequisites, policies, regulations)
- **Maintain organizational context** (business rules, schedules, procedures)
- **Search efficiently** using semantic vector search across all knowledge types

Long-term memory is a flexible storage mechanism: user-scoped memories enable personalization ("Student prefers online courses"), while application-scoped memories provide consistent behavior for everyone ("CS401 requires CS201", "Registration opens 2 weeks before semester").

### How It Works

```
Session 1: User shares preferences → Store in long-term memory
Session 2: User asks for recommendations → Search memory → Personalized response
Session 3: User updates preferences → Update memory accordingly
```

---

## Three Types of Long-term Memory

The Agent Memory Server supports three distinct memory types, each optimized for different kinds of information:

### 1. Semantic Memory - Facts and Knowledge

**Purpose:** Store timeless facts, preferences, and knowledge independent of when they were learned. Can be user-scoped (personalization) or application-scoped (domain knowledge).

**User-Scoped Examples:**
- "Student's major is Computer Science"
- "Student prefers online courses"
- "Student wants to graduate in Spring 2026"

**Application-Scoped Examples:**
- "CS401 requires CS201 and MATH301 as prerequisites"
- "Online courses have asynchronous discussion forums"
- "Maximum file upload size for assignments is 50MB"

**When to use:** Information that remains true regardless of time context.

---

### 2. Episodic Memory - Events and Experiences

**Purpose:** Store time-bound events and experiences where sequence matters.

**Examples:**
- "Student enrolled in CS101 on 2024-09-15"
- "Student completed CS101 with grade A on 2024-12-10"
- "Student asked about machine learning courses on 2024-09-20"

**When to use:** Timeline-based information where timing or sequence is important.

---

### 3. Message Memory - Context-Rich Conversations

**Purpose:** Store full conversation snippets where complete context is crucial.

**Examples:**
- Detailed career planning discussion with nuanced advice
- Professor's specific guidance about research opportunities
- Student's explanation of personal learning challenges

**When to use:** When summary would lose important nuance, tone, or exact wording.

**⚠️ Use sparingly** - Message memories are token-expensive!

---

## 🎯 Choosing the Right Memory Type

### Decision Framework

**Ask yourself these questions:**

1. **Can you extract a simple fact?** → Use **Semantic**
2. **Does timing matter?** → Use **Episodic**
3. **Is full context crucial?** → Use **Message** (rarely)

**Default strategy: Prefer Semantic** - they're compact, searchable, and efficient.

### Quick Reference Table

| Information Type | Memory Type | Example |
|-----------------|-------------|----------|
| Preference | Semantic | "Prefers morning classes" |
| Fact | Semantic | "Major is Computer Science" |
| Goal | Semantic | "Wants to graduate in 2026" |
| Event | Episodic | "Enrolled in CS401 on 2024-09-15" |
| Timeline | Episodic | "Completed CS101, then CS201" |
| Complex discussion | Message | [Full career planning conversation] |

---

## 🧪 Hands-On: Long-term Memory in Action

Let's put these concepts into practice with code examples.


### Setup: Student ID for Long-term Memory

Long-term memories are user-scoped, so we need a student ID.


In [26]:
# Setup for long-term memory demo
lt_student_id = "sarah_chen"

print(f"""🎯 Long-term Memory Demo Setup
   Student ID: {lt_student_id}
   Ready to store and search persistent memories""")

🎯 Long-term Memory Demo Setup
   Student ID: sarah_chen
   Ready to store and search persistent memories


### Step 1: Store Semantic Memories (Facts)

Semantic memories are timeless facts about the student. Let's store several facts about Sarah's preferences and academic status.


In [27]:
if MEMORY_SERVER_AVAILABLE:
    print("=" * 80)
    print("📍 STEP 1: Storing Semantic Memories (Facts)")
    print("=" * 80)

    # Define semantic memories (timeless facts)
    semantic_memories = [
        "Student prefers online courses over in-person classes",
        "Student's major is Computer Science with focus on AI/ML",
        "Student wants to graduate in Spring 2026",
        "Student prefers morning classes, no classes on Fridays",
        "Student has completed Introduction to Programming and Data Structures",
        "Student is currently taking Linear Algebra",
    ]
    print(f"\n📝 Storing {len(semantic_memories)} semantic memories...")

    # Store each semantic memory
    for memory_text in semantic_memories:
        memory_record = ClientMemoryRecord(
            text=memory_text,
            user_id=lt_student_id,
            memory_type="semantic",
            topics=["preferences", "academic_info"],
        )
        await memory_client.create_long_term_memory([memory_record])
        print(f"   ✅ {memory_text}")

    print(f"""
✅ Stored {len(semantic_memories)} semantic memories
   Memory type: semantic (timeless facts)
   Topics: preferences, academic_info""")

📍 STEP 1: Storing Semantic Memories (Facts)

📝 Storing 6 semantic memories...
19:17:12 httpx INFO   HTTP Request: POST http://localhost:8088/v1/long-term-memory/ "HTTP/1.1 200 OK"


   ✅ Student prefers online courses over in-person classes
19:17:12 httpx INFO   HTTP Request: POST http://localhost:8088/v1/long-term-memory/ "HTTP/1.1 200 OK"


   ✅ Student's major is Computer Science with focus on AI/ML
19:17:12 httpx INFO   HTTP Request: POST http://localhost:8088/v1/long-term-memory/ "HTTP/1.1 200 OK"


   ✅ Student wants to graduate in Spring 2026
19:17:12 httpx INFO   HTTP Request: POST http://localhost:8088/v1/long-term-memory/ "HTTP/1.1 200 OK"


   ✅ Student prefers morning classes, no classes on Fridays
19:17:12 httpx INFO   HTTP Request: POST http://localhost:8088/v1/long-term-memory/ "HTTP/1.1 200 OK"


   ✅ Student has completed Introduction to Programming and Data Structures
19:17:12 httpx INFO   HTTP Request: POST http://localhost:8088/v1/long-term-memory/ "HTTP/1.1 200 OK"


   ✅ Student is currently taking Linear Algebra

✅ Stored 6 semantic memories
   Memory type: semantic (timeless facts)
   Topics: preferences, academic_info


### What We Just Did: Semantic Memories

**Stored 6 semantic memories:**
- Student preferences (online courses, morning classes)
- Academic information (major, graduation date)
- Course history (completed, current)

**Why semantic?**
- These are timeless facts
- No specific date/time context needed
- Compact and efficient

**How they're stored:**
- Vector-indexed for semantic search
- Tagged with topics for organization
- Automatically deduplicated

---


### Step 2: Store Episodic Memories (Events)

Episodic memories are time-bound events. Let's store some events from Sarah's academic timeline.


In [28]:
if MEMORY_SERVER_AVAILABLE:
    print("\n" + "=" * 80)
    print("📍 STEP 2: Storing Episodic Memories (Events)")
    print("=" * 80)

    # Define episodic memories (time-bound events)
    episodic_memories = [
        "Student enrolled in Introduction to Programming on 2024-09-01",
        "Student completed Introduction to Programming with grade A on 2024-12-15",
        "Student asked about machine learning courses on 2024-09-20",
    ]

    print(f"\n📝 Storing {len(episodic_memories)} episodic memories...")

    # Store each episodic memory
    for memory_text in episodic_memories:
        memory_record = ClientMemoryRecord(
            text=memory_text,
            user_id=lt_student_id,
            memory_type="episodic",
            topics=["enrollment", "courses"],
        )
        await memory_client.create_long_term_memory([memory_record])
        print(f"   ✅ {memory_text}")

    print(f"""
✅ Stored {len(episodic_memories)} episodic memories
   Memory type: episodic (time-bound events)
   Topics: enrollment, courses""")


📍 STEP 2: Storing Episodic Memories (Events)

📝 Storing 3 episodic memories...
19:17:12 httpx INFO   HTTP Request: POST http://localhost:8088/v1/long-term-memory/ "HTTP/1.1 200 OK"


   ✅ Student enrolled in Introduction to Programming on 2024-09-01
19:17:12 httpx INFO   HTTP Request: POST http://localhost:8088/v1/long-term-memory/ "HTTP/1.1 200 OK"


   ✅ Student completed Introduction to Programming with grade A on 2024-12-15
19:17:12 httpx INFO   HTTP Request: POST http://localhost:8088/v1/long-term-memory/ "HTTP/1.1 200 OK"


   ✅ Student asked about machine learning courses on 2024-09-20

✅ Stored 3 episodic memories
   Memory type: episodic (time-bound events)
   Topics: enrollment, courses


### What We Just Did: Episodic Memories

**Stored 3 episodic memories:**
- Enrollment event (Introduction to Programming on 2024-09-01)
- Completion event (Introduction to Programming with grade A on 2024-12-15)
- Interaction event (asked about ML courses on 2024-09-20)

**Why episodic?**
- These are time-bound events
- Timing and sequence matter
- Captures academic timeline

**Difference from semantic:**
- Semantic: "Student has completed Introduction to Programming" (timeless fact)
- Episodic: "Student completed Introduction to Programming with grade A on 2024-12-15" (specific event)

---


### Step 3: Search Long-term Memory

Now let's search our long-term memories using natural language queries. The system will use semantic search to find relevant memories.


In [29]:
if MEMORY_SERVER_AVAILABLE:
    from agent_memory_client.filters import UserId

    print("\n" + "=" * 80)
    print("📍 STEP 3: Searching Long-term Memory")
    print("=" * 80)

    # Query 1: What does the student prefer?
    search_query_1 = "What does the student prefer?"
    print(f"\n🔍 Query: '{search_query_1}'")

    search_results_1 = await memory_client.search_long_term_memory(
        text=search_query_1, user_id=UserId(eq=lt_student_id), limit=3
    )

    if search_results_1.memories:
        print(f"   📚 Found {len(search_results_1.memories)} relevant memories:")
        for i, memory in enumerate(search_results_1.memories[:3], 1):
            print(f"      {i}. {memory.text}")
    else:
        print("   ⚠️  No memories found")

    # Query 2: What courses has the student completed?
    search_query_2 = "What courses has the student completed?"
    print(f"\n🔍 Query: '{search_query_2}'")

    search_results_2 = await memory_client.search_long_term_memory(
        text=search_query_2, user_id=UserId(eq=lt_student_id), limit=5
    )

    if search_results_2.memories:
        print(f"   📚 Found {len(search_results_2.memories)} relevant memories:")
        for i, memory in enumerate(search_results_2.memories[:5], 1):
            print(f"      {i}. {memory.text}")
    else:
        print("   ⚠️  No memories found")

    # Query 3: What is the student's major?
    search_query_3 = "What is the student's major?"
    print(f"\n🔍 Query: '{search_query_3}'")

    search_results_3 = await memory_client.search_long_term_memory(
        text=search_query_3, user_id=UserId(eq=lt_student_id), limit=3
    )

    if search_results_3.memories:
        print(f"   📚 Found {len(search_results_3.memories)} relevant memories:")
        for i, memory in enumerate(search_results_3.memories[:3], 1):
            print(f"      {i}. {memory.text}")
    else:
        print("   ⚠️  No memories found")

    print("\n" + "=" * 80)
    print("✅ DEMO COMPLETE: Long-term memory enables persistent knowledge!")
    print("=" * 80)
else:
    print("⚠️  Memory Server not available. Skipping demo.")


📍 STEP 3: Searching Long-term Memory

🔍 Query: 'What does the student prefer?'


19:17:12 httpx INFO   HTTP Request: POST http://localhost:8088/v1/long-term-memory/search?optimize_query=false "HTTP/1.1 200 OK"


   ⚠️  No memories found

🔍 Query: 'What courses has the student completed?'


19:17:13 httpx INFO   HTTP Request: POST http://localhost:8088/v1/long-term-memory/search?optimize_query=false "HTTP/1.1 200 OK"


   📚 Found 1 relevant memories:
      1. Student has completed Introduction to Programming and Data Structures

🔍 Query: 'What is the student's major?'


19:17:13 httpx INFO   HTTP Request: POST http://localhost:8088/v1/long-term-memory/search?optimize_query=false "HTTP/1.1 200 OK"


   📚 Found 3 relevant memories:
      1. Student's major is Computer Science with focus on AI/ML
      2. Student enrolled in Introduction to Programming on 2024-09-01
      3. Student has completed Introduction to Programming and Data Structures

✅ DEMO COMPLETE: Long-term memory enables persistent knowledge!


### 🎯 Long-term Memory Demo Summary

**📊 What We Did:**
- **Step 1:** Stored 6 semantic memories (facts) - preferences, major, graduation date
- **Step 2:** Stored 3 episodic memories (events) - enrollment, completion, interaction
- **Step 3:** Searched long-term memory with natural language queries

**✅ Key Benefits:**
- Persistent knowledge across sessions
- Semantic search (not keyword matching)
- Automatic deduplication
- Topic-based organization

**💡 Key Insight:**
Long-term memory enables personalization and knowledge accumulation across sessions. It's the foundation for building agents that remember and learn from users.

### Key Insight: User Context Type

Long-term memory provides part of the **User Context** - the second context type from Module 1:

1. **System Context** - Role and instructions (static)
2. **User Context** - Profile + long-term memories (dynamic, user-specific) ← **Long-term memories contribute here!**
3. **Conversation Context** - Working memory (dynamic, session-specific)
4. **Retrieved Context** - RAG results (dynamic, query-specific)

Long-term memories enhance User Context by adding persistent knowledge about the user's preferences, history, and goals.

---

## 🔍 Understanding Memory Search: Why Semantic Only?

You might have noticed that the Agent Memory Server uses **semantic (vector) search only** - no keyword search or hybrid search. Let's understand why this is the right choice.

### Memory vs. Course Catalog: Different Search Needs

| Aspect | Memory Search | Course Catalog Search |
|--------|---------------|----------------------|
| **Data Type** | Conversational facts | Structured catalog |
| **Content** | "Student prefers online courses"<br>"Completed CS101 last semester" | Course codes, titles, syllabi<br>Departments, prerequisites |
| **Queries** | "What does the student prefer?"<br>"What courses has the student taken?" | "CS101"<br>"beginner programming courses" |
| **Exact Matches?** | ❌ No codes/IDs to match | ✅ Course codes, departments |
| **Best Search** | **Semantic only** | **Hybrid (semantic + keyword)** |

### Why Semantic Search for Memories?

**1. Conversational Content**
```python
# Memory content is natural language
memories = [
    "Student prefers online courses over in-person",
    "Interested in machine learning and AI",
    "Completed CS101 with grade A last semester"
]

# Queries are also natural language
query = "What does the student prefer?"
# ✅ Semantic search finds: "Student prefers online courses..."
# ❌ Keyword search would miss it (no exact word "prefer" in memory)
```

**2. No Exact Codes/IDs**
```python
# Memories don't have exact codes to match
memory = "Student prefers online courses"  # No "ONLINE-001" code

# vs. Course catalog
course = {
    "course_code": "CS101",  # ← Exact code for keyword search
    "department": "Computer Science",  # ← Exact category
    "title": "Introduction to Programming"
}
```

**3. Small Dataset Per User**
```python
# Typical user has <100 memories
# Vector search is fast enough
# No need for keyword optimization

# vs. Course catalog with 1000s of courses
# Hybrid search improves performance and precision
```

### Real-World Example

**Memory Search (Semantic):**
```python
# Query: "What are the student's interests?"
results = await memory_client.search_long_term_memory(
    text="What are the student's interests?",
    user_id=UserId(eq=student_id)
)
# Finds: "Interested in machine learning and AI"
#        "Enjoys data science projects"
# ✅ Semantic understanding matches conceptually
```

**Course Search (Hybrid):**
```python
# Query: "beginner CS programming courses"
# Semantic: Finds conceptually similar courses
# Keyword: Filters by department="Computer Science", difficulty="Beginner"
# Hybrid: Best of both worlds!
```

### When Would Memory Need Hybrid Search?

You'd add keyword/hybrid search to memories if:
- ❌ Memories contained exact codes/IDs to match
- ❌ Users searched for specific technical terms
- ❌ Dataset was huge (millions of memories per user)

**But:** None of these apply to conversational memory!

### Key Takeaway

**Different data types need different search strategies:**

```
Conversational Data (Memories)
    ↓
Natural language content
    ↓
Semantic search only ✅

Structured Catalog (Courses)
    ↓
Codes + descriptions + metadata
    ↓
Hybrid search (semantic + keyword) ✅

Reference Data (Course Details)
    ↓
Fetched by ID only
    ↓
No search needed (plain keys) ✅
```

This is why Module 2 teaches all search types, but Module 4 uses semantic-only for memories!

---

## 🏷️ Advanced: Topics and Filtering

Topics help organize and filter memories. Let's explore how to use them effectively.


In [30]:
if MEMORY_SERVER_AVAILABLE:
    topics_student_id = "sarah_chen"

    print("=" * 80)
    print("🏷️  TOPICS AND FILTERING DEMO")
    print("=" * 80)

    print("\n📍 Storing Memories with Topics")
    print("-" * 80)

    # Define memories with their topics
    memories_with_topics = [
        ("Student prefers online courses", ["preferences", "course_format"]),
        ("Student's major is Computer Science", ["academic_info", "major"]),
        ("Student wants to graduate in Spring 2026", ["goals", "graduation"]),
        ("Student prefers morning classes", ["preferences", "schedule"]),
    ]

    # Store each memory
    for memory_text, topics in memories_with_topics:
        memory_record = ClientMemoryRecord(
            text=memory_text,
            user_id=topics_student_id,
            memory_type="semantic",
            topics=topics,
        )
        await memory_client.create_long_term_memory([memory_record])
        print(f"   ✅ {memory_text}")
        print(f"      Topics: {', '.join(topics)}")

🏷️  TOPICS AND FILTERING DEMO

📍 Storing Memories with Topics
--------------------------------------------------------------------------------
19:17:13 httpx INFO   HTTP Request: POST http://localhost:8088/v1/long-term-memory/ "HTTP/1.1 200 OK"


   ✅ Student prefers online courses
      Topics: preferences, course_format
19:17:13 httpx INFO   HTTP Request: POST http://localhost:8088/v1/long-term-memory/ "HTTP/1.1 200 OK"


   ✅ Student's major is Computer Science
      Topics: academic_info, major
19:17:13 httpx INFO   HTTP Request: POST http://localhost:8088/v1/long-term-memory/ "HTTP/1.1 200 OK"


   ✅ Student wants to graduate in Spring 2026
      Topics: goals, graduation
19:17:13 httpx INFO   HTTP Request: POST http://localhost:8088/v1/long-term-memory/ "HTTP/1.1 200 OK"


   ✅ Student prefers morning classes
      Topics: preferences, schedule


### Filter memories by type


In [31]:
if MEMORY_SERVER_AVAILABLE:
    print("\n📍 Filtering by Memory Type: Semantic")
    print("-" * 80)

    from agent_memory_client.filters import MemoryType, UserId

    # Search for all semantic memories
    results = await memory_client.search_long_term_memory(
        text="",  # Empty query returns all
        user_id=UserId(eq=topics_student_id),
        memory_type=MemoryType(eq="semantic"),
        limit=10,
    )

    print(f"   Found {len(results.memories)} semantic memories:")
    for i, memory in enumerate(results.memories[:5], 1):
        topics_str = ", ".join(memory.topics) if memory.topics else "none"
        print(f"   {i}. {memory.text}")
        print(f"      Topics: {topics_str}")

    print("\n" + "=" * 80)
    print("✅ Topics enable organized, filterable memory management!")
    print("=" * 80)


📍 Filtering by Memory Type: Semantic
--------------------------------------------------------------------------------


19:17:13 httpx INFO   HTTP Request: POST http://localhost:8088/v1/long-term-memory/search?optimize_query=false "HTTP/1.1 200 OK"


   Found 5 semantic memories:
   1. Student is currently taking Linear Algebra
      Topics: preferences, academic_info
   2. Student prefers online courses over in-person classes
      Topics: preferences, academic_info
   3. Student prefers morning classes, no classes on Fridays
      Topics: preferences, academic_info
   4. Student's major is Computer Science with focus on AI/ML
      Topics: preferences, academic_info
   5. Student has completed Introduction to Programming and Data Structures
      Topics: preferences, academic_info

✅ Topics enable organized, filterable memory management!


### 🎯 Why Topics Matter

**Organization:**
- Group related memories together
- Easy to find memories by category

**Filtering:**
- Search within specific topics
- Filter by memory type (semantic, episodic, message)

**Best Practices:**
- Use consistent topic names
- Keep topics broad enough to be useful
- Common topics: `preferences`, `academic_info`, `goals`, `schedule`, `courses`

---

## 🔄 Cross-Session Memory Persistence

Let's verify that memories persist across sessions.


In [32]:
if MEMORY_SERVER_AVAILABLE:
    cross_session_student_id = "sarah_chen"

    print("=" * 80)
    print("🔄 CROSS-SESSION MEMORY PERSISTENCE DEMO")
    print("=" * 80)

    print("\n📍 SESSION 1: Storing Memories")
    print("-" * 80)

    memory_record = ClientMemoryRecord(
        text="Student is interested in machine learning and AI",
        user_id=cross_session_student_id,
        memory_type="semantic",
        topics=["interests", "AI"],
    )
    await memory_client.create_long_term_memory([memory_record])
    print("   ✅ Stored: Student is interested in machine learning and AI")

    print("\n📍 SESSION 2: New Session, Same Student")
    print("-" * 80)

    # Create a new memory client (simulating a new session)
    new_session_config = MemoryClientConfig(
        base_url=AGENT_MEMORY_URL,
        default_namespace="redis_university",
    )
    new_session_client = MemoryAPIClient(config=new_session_config)

    print("   🔄 New session started for the same student")

    print("\n   🔍 Searching: 'What are the student's interests?'")
    cross_session_results = await new_session_client.search_long_term_memory(
        text="What are the student's interests?",
        user_id=UserId(eq=cross_session_student_id),
        limit=3,
    )

    if cross_session_results.memories:
        print(f"\n   ✅ Memories accessible from new session:")
        for i, memory in enumerate(cross_session_results.memories[:3], 1):
            print(f"      {i}. {memory.text}")
    else:
        print("   ⚠️  No memories found")

    print("\n" + "=" * 80)
    print("✅ Long-term memories persist across sessions!")
    print("=" * 80)

🔄 CROSS-SESSION MEMORY PERSISTENCE DEMO

📍 SESSION 1: Storing Memories
--------------------------------------------------------------------------------


19:17:13 httpx INFO   HTTP Request: POST http://localhost:8088/v1/long-term-memory/ "HTTP/1.1 200 OK"


   ✅ Stored: Student is interested in machine learning and AI

📍 SESSION 2: New Session, Same Student
--------------------------------------------------------------------------------


   🔄 New session started for the same student

   🔍 Searching: 'What are the student's interests?'


19:17:13 httpx INFO   HTTP Request: POST http://localhost:8088/v1/long-term-memory/search?optimize_query=false "HTTP/1.1 200 OK"



   ✅ Memories accessible from new session:
      1. Student's major is Computer Science with focus on AI/ML
      2. Student prefers morning classes, no classes on Fridays
      3. Student prefers online courses over in-person classes

✅ Long-term memories persist across sessions!


### 🎯 Cross-Session Persistence

**What We Demonstrated:**
- **Session 1:** Stored memories about student interests
- **Session 2:** Created new client (simulating new session)
- **Result:** Memories from Session 1 are accessible in Session 2

**Why This Matters:**
- Users don't have to repeat themselves
- Personalization works across days, weeks, months
- Knowledge accumulates over time

**Contrast with Working Memory:**
- Working memory: Session-scoped (persists within the session, like ChatGPT conversations)
- Long-term memory: User-scoped (persists across all sessions indefinitely)


---

## 📚 Part 3: Memory-Enhanced RAG

Now let's combine everything we've learned: working memory, long-term memory, and the hierarchical RAG system from Module 2.

### The Complete Pattern

```
1. Load working memory (conversation history)
2. Search long-term memory (user facts)
3. Hierarchical RAG search (summaries → details)
4. Assemble all four context types
5. Generate response
6. Save working memory (updated conversation)
```

This gives us **stateful, personalized, context-aware conversations**.

---

## 🚫 Before: Stateless RAG (Module 2 Approach)

Let's first recall how Module 2's stateless RAG worked, and see its limitations.


In [33]:
print("=" * 80)
print("🚫 STATELESS RAG DEMO")
print("=" * 80)

stateless_query_1 = "I'm interested in machine learning courses"
print(f"\n👤 User: {stateless_query_1}\n")

# Search courses using hierarchical retrieval
stateless_summaries, stateless_details = await hierarchical_manager.hierarchical_search(
    query=stateless_query_1,
    summary_limit=3,
    detail_limit=2
)

# Assemble context (System + User + Retrieved only - NO conversation history)
stateless_system_prompt = """You are a Redis University course advisor.

CRITICAL RULES:
- ONLY discuss and recommend courses from the "Relevant Courses" list provided below
- Do NOT mention, suggest, or make up any courses that are not in the provided list
- If the available courses don't perfectly match the request, recommend the best options from what IS available"""

stateless_user_context = f"""Student: {sarah.name}
Major: {sarah.major}
Interests: {', '.join(sarah.interests)}
Completed: {', '.join(sarah.completed_courses)}
"""

# Use context assembler for progressive disclosure
stateless_retrieved_context = context_assembler.assemble_hierarchical_context(
    summaries=stateless_summaries,
    details=stateless_details,
    query=stateless_query_1
)

# Generate response
stateless_messages_1 = [
    SystemMessage(content=stateless_system_prompt),
    HumanMessage(
        content=f"{stateless_user_context}\n\n{stateless_retrieved_context}\n\nQuery: {stateless_query_1}"
    ),
]

stateless_response_1 = llm.invoke(stateless_messages_1).content
print(f"🤖 Agent: {stateless_response_1}")

# ❌ No conversation history stored
# ❌ Next query won't remember this interaction

🚫 STATELESS RAG DEMO

👤 User: I'm interested in machine learning courses

19:17:13 redis_context_course.hierarchical_manager INFO   Hierarchical search: 'I'm interested in machine learning courses' (summaries=3, details=2)


19:17:14 httpx INFO   HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


19:17:14 redis_context_course.hierarchical_manager INFO   Found 0 course summaries for query: I'm interested in machine learning courses


19:17:14 redis_context_course.hierarchical_manager INFO   Fetched 0 course details


19:17:14 redis_context_course.hierarchical_manager INFO   Hierarchical search complete: 0 summaries, 0 details


19:17:16 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


🤖 Agent: It seems there are no direct matches for machine learning courses in the current list. However, I can recommend some courses that might still be beneficial for your interests in data science and algorithms. Here are some relevant courses you might consider:

1. **RU101: Introduction to Redis Data Structures** - This course will provide a solid foundation in understanding how data is structured and managed, which is crucial for data science.

2. **RU102: Redis for Developers** - While not specifically about machine learning, this course will enhance your skills in using Redis, which can be a valuable tool in data-driven applications.

3. **RU202: Redis Streams** - This course focuses on real-time data processing, which is an important aspect of data science and can be useful in machine learning applications.

These courses can help build a strong foundation in data management and processing, which are key components in the field of data science and machine learning.


### Query 2: Follow-up with pronoun reference (fails)

Now let's try a follow-up that requires conversation history.


In [34]:
stateless_query_2 = "What are the prerequisites for the first one?"
print(f"👤 User: {stateless_query_2}")
print("   Note: 'the first one' refers to the first course from Query 1\n")

# Search courses (will search for "prerequisites first one" - not helpful)
stateless_summaries_2, stateless_details_2 = await hierarchical_manager.hierarchical_search(
    query=stateless_query_2,
    summary_limit=3,
    detail_limit=2
)

# Assemble context (NO conversation history from Query 1)
stateless_retrieved_context_2 = context_assembler.assemble_hierarchical_context(
    summaries=stateless_summaries_2,
    details=stateless_details_2,
    query=stateless_query_2
)

# Generate response
stateless_messages_2 = [
    SystemMessage(content=stateless_system_prompt),
    HumanMessage(
        content=f"{stateless_user_context}\n\n{stateless_retrieved_context_2}\n\nQuery: {stateless_query_2}"
    ),
]

stateless_response_2 = llm.invoke(stateless_messages_2).content
print(f"\n🤖 Agent: {stateless_response_2}")
print("\n❌ Agent can't resolve 'the first one' - no conversation history!")

👤 User: What are the prerequisites for the first one?
   Note: 'the first one' refers to the first course from Query 1

19:17:16 redis_context_course.hierarchical_manager INFO   Hierarchical search: 'What are the prerequisites for the first one?' (summaries=3, details=2)


19:17:16 httpx INFO   HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


19:17:16 redis_context_course.hierarchical_manager INFO   Found 0 course summaries for query: What are the prerequisites for the first one?


19:17:16 redis_context_course.hierarchical_manager INFO   Fetched 0 course details


19:17:16 redis_context_course.hierarchical_manager INFO   Hierarchical search complete: 0 summaries, 0 details


19:17:21 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



🤖 Agent: It seems there are no relevant courses found based on your query. However, I can recommend some courses that might align with Sarah's interests in machine learning, data science, and algorithms. Here are some options from the "Relevant Courses" list:

1. **RU101: Introduction to Redis Data Structures** - This course provides a foundational understanding of Redis, which can be beneficial for data science applications.

2. **RU102: Redis for Developers** - This course is suitable for developers looking to integrate Redis into their applications, which can be useful for building efficient algorithms.

3. **RU202: Redis Streams** - This course focuses on Redis Streams, which can be useful for handling real-time data, a key component in data science and machine learning.

These courses do not have specific prerequisites listed, but a background in computer science should be beneficial. If you have any other questions or need further assistance, feel free to ask!

❌ Agent can't res

### 🎯 What Just Happened?

**Query 1:** "I'm interested in machine learning courses"
- ✅ Works fine - searches and returns ML courses

**Query 2:** "What are the prerequisites for **the first one**?"
- ❌ **Fails** - Agent doesn't know what "the first one" refers to
- ❌ No conversation history stored
- ❌ Each query is completely independent

**The Problem:** Natural conversation requires context from previous turns.

---

## ✅ After: Memory-Enhanced RAG

Now let's add memory to enable natural conversations.

### Helper Function: Memory-Enhanced RAG with Hierarchical Retrieval

This function combines all four context types with hierarchical retrieval.


In [35]:
async def memory_enhanced_rag_query(
    user_query: str,
    student_profile: StudentProfile,
    session_id: str,
    summary_limit: int = 3,
    detail_limit: int = 2
) -> str:
    """Generate response using memory-enhanced RAG with hierarchical retrieval"""

    if not MEMORY_SERVER_AVAILABLE:
        return "⚠️ Memory Server not available"

    from agent_memory_client.filters import UserId

    student_id = student_profile.email.split("@")[0]

    # 1. Load working memory (conversation history)
    _, working_memory = await memory_client.get_or_create_working_memory(
        session_id=session_id, user_id=student_id, model_name="gpt-4o"
    )

    # Build conversation messages
    conversation_messages = []
    for msg in working_memory.messages:
        if msg.role == "user":
            conversation_messages.append(HumanMessage(content=msg.content))
        elif msg.role == "assistant":
            conversation_messages.append(AIMessage(content=msg.content))

    # 2. Search long-term memory (user facts)
    longterm_results = await memory_client.search_long_term_memory(
        text=user_query, user_id=UserId(eq=student_id), limit=5
    )
    longterm_memories = (
        [m.text for m in longterm_results.memories] if longterm_results.memories else []
    )

    # 3. Hierarchical RAG search (summaries → details)
    summaries, details = await hierarchical_manager.hierarchical_search(
        query=user_query,
        summary_limit=summary_limit,
        detail_limit=detail_limit
    )

    # 4. Assemble all four context types
    # System Context
    system_prompt = """You are a Redis University course advisor.

Your role:
- Help students find and enroll in courses from our catalog
- Provide personalized recommendations based on available courses
- Answer questions about courses, prerequisites, schedules

CRITICAL RULES:
- You can ONLY recommend courses that appear in the "Relevant Courses" list below
- Do NOT suggest courses that are not in the "Relevant Courses" list
- Use conversation history to resolve references ("it", "that course", "the first one")
- Use long-term memories to personalize your recommendations
- Be helpful, supportive, and encouraging"""

    # User Context (profile + long-term memories)
    user_context = f"""Student Profile:
- Name: {student_profile.name}
- Major: {student_profile.major}
- Year: {student_profile.year}
- Interests: {', '.join(student_profile.interests)}
- Completed: {', '.join(student_profile.completed_courses)}
- Current: {', '.join(student_profile.current_courses)}
- Preferred Format: {student_profile.preferred_format.value}
- Preferred Difficulty: {student_profile.preferred_difficulty.value}"""

    if longterm_memories:
        user_context += f"\n\nLong-term Memories:\n" + "\n".join(
            [f"- {m}" for m in longterm_memories]
        )

    # Retrieved Context (hierarchical)
    retrieved_context = context_assembler.assemble_hierarchical_context(
        summaries=summaries,
        details=details,
        query=user_query
    )

    # 5. Build messages and generate response
    messages = [SystemMessage(content=system_prompt)]
    messages.extend(conversation_messages)  # Conversation Context
    messages.append(
        HumanMessage(
            content=f"{user_context}\n\n{retrieved_context}\n\nQuery: {user_query}"
        )
    )

    response = llm.invoke(messages).content

    # 6. Save working memory (updated conversation)
    working_memory.messages.extend(
        [
            MemoryMessage(role="user", content=user_query),
            MemoryMessage(role="assistant", content=response),
        ]
    )
    await memory_client.put_working_memory(
        session_id=session_id,
        memory=working_memory,
        user_id=student_id,
        model_name="gpt-4o",
    )

    return response


print("✅ Memory-enhanced RAG function created")
print("   Uses: Working memory + Long-term memory + Hierarchical RAG")

✅ Memory-enhanced RAG function created
   Uses: Working memory + Long-term memory + Hierarchical RAG


---

## 🧪 Complete Demo: Memory-Enhanced RAG

Now let's test the complete system with a multi-turn conversation.


In [36]:
# Set up demo session
demo_session_id = f"complete_demo_{uuid.uuid4().hex[:8]}"

print("=" * 80)
print("🧪 MEMORY-ENHANCED RAG DEMO")
print("=" * 80)
print(f"\n👤 Student: {sarah.name}")
print(f"📧 Session: {demo_session_id}")

🧪 MEMORY-ENHANCED RAG DEMO

👤 Student: Sarah Chen
📧 Session: complete_demo_6ac53750


### Turn 1: Initial Query


In [37]:
print("\n" + "=" * 80)
print("📍 TURN 1: Initial Query")
print("=" * 80)

demo_query_1 = "I'm interested in machine learning courses"
print(f"\n👤 User: {demo_query_1}")

demo_response_1 = await memory_enhanced_rag_query(demo_query_1, sarah, demo_session_id)

print(f"\n🤖 Agent: {demo_response_1}")
print("\n✅ Conversation saved to working memory")


📍 TURN 1: Initial Query

👤 User: I'm interested in machine learning courses
19:17:21 httpx INFO   HTTP Request: GET http://localhost:8088/v1/working-memory/complete_demo_6ac53750?user_id=sarah.chen&namespace=redis_university&model_name=gpt-4o "HTTP/1.1 404 Not Found"


19:17:21 httpx INFO   HTTP Request: PUT http://localhost:8088/v1/working-memory/complete_demo_6ac53750?user_id=sarah.chen&model_name=gpt-4o "HTTP/1.1 200 OK"


19:17:21 httpx INFO   HTTP Request: POST http://localhost:8088/v1/long-term-memory/search?optimize_query=false "HTTP/1.1 200 OK"


19:17:21 redis_context_course.hierarchical_manager INFO   Hierarchical search: 'I'm interested in machine learning courses' (summaries=3, details=2)


19:17:22 httpx INFO   HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


...

19:17:22 redis_context_course.hierarchical_manager INFO   Fetched 0 course details


19:17:22 redis_context_course.hierarchical_manager INFO   Hierarchical search complete: 0 summaries, 0 details


19:17:24 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


19:17:24 httpx INFO   HTTP Request: PUT http://localhost:8088/v1/working-memory/complete_demo_6ac53750?user_id=sarah.chen&model_name=gpt-4o "HTTP/1.1 200 OK"



🤖 Agent: Hi Sarah! It looks like we don't have any machine learning courses available at the moment. However, I can recommend some other courses that might align with your interests in data science and algorithms. Let's see what we have:

### Relevant Courses:
1. **Data Structures and Algorithms (CS301)**
   - Format: Online
   - Difficulty: Intermediate
   - Description: Dive deeper into data structures and algorithms, building on what you've learned in CS201.

2. **Introduction to Data Science (DS101)**
   - Format: Online
   - Difficulty: Intermediate
   - Description: Explore the basics of data science, including data manipulation and visualization techniques.

Given your background and interests, "Data Structures and Algorithms (CS301)" could be a great fit to further enhance your algorithm skills. Additionally, "Introduction to Data Science (DS101)" would be a good starting point to delve into data science concepts.

Let me know if you would like more information on any of these

### Turn 2: Follow-up with Pronoun Reference

Now let's ask about "the first one" - a reference that requires conversation history.


In [38]:
print("\n" + "=" * 80)
print("📍 TURN 2: Follow-up with Pronoun Reference")
print("=" * 80)

demo_query_2 = "What are the prerequisites for the first one?"
print(f"\n👤 User: {demo_query_2}")
print("   Note: 'the first one' refers to the first course mentioned in Turn 1")

demo_response_2 = await memory_enhanced_rag_query(demo_query_2, sarah, demo_session_id)

print(f"\n🤖 Agent: {demo_response_2}")
print("\n✅ Agent resolved 'the first one' using conversation history!")


📍 TURN 2: Follow-up with Pronoun Reference

👤 User: What are the prerequisites for the first one?
   Note: 'the first one' refers to the first course mentioned in Turn 1
19:17:24 httpx INFO   HTTP Request: GET http://localhost:8088/v1/working-memory/complete_demo_6ac53750?user_id=sarah.chen&namespace=redis_university&model_name=gpt-4o "HTTP/1.1 200 OK"


19:17:25 httpx INFO   HTTP Request: POST http://localhost:8088/v1/long-term-memory/search?optimize_query=false "HTTP/1.1 200 OK"


19:17:25 redis_context_course.hierarchical_manager INFO   Hierarchical search: 'What are the prerequisites for the first one?' (summaries=3, details=2)


19:17:25 httpx INFO   HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


19:17:25 redis_context_course.hierarchical_manager INFO   Found 0 course summaries for query: What are the prerequisites for the first one?


19:17:25 redis_context_course.hierarchical_manager INFO   Fetched 0 course details


19:17:25 redis_context_course.hierarchical_manager INFO   Hierarchical search complete: 0 summaries, 0 details


19:17:27 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


19:17:27 httpx INFO   HTTP Request: PUT http://localhost:8088/v1/working-memory/complete_demo_6ac53750?user_id=sarah.chen&model_name=gpt-4o "HTTP/1.1 200 OK"



🤖 Agent: Hi Sarah! It seems like there was a bit of confusion with the search results. Based on your interests and completed courses, I previously recommended "Data Structures and Algorithms (CS301)" and "Introduction to Data Science (DS101)" as potential courses for you.

### Prerequisites:

1. **Data Structures and Algorithms (CS301)**
   - Prerequisite: Completion of CS201 (which you've already completed)

2. **Introduction to Data Science (DS101)**
   - Prerequisite: None specified, but a basic understanding of programming and data manipulation is helpful.

Both courses are offered online and are at an intermediate level, which matches your preferences. If you have any more questions or need further assistance, feel free to ask!

✅ Agent resolved 'the first one' using conversation history!


### Turn 3: Another Follow-up

Let's ask if the student meets the prerequisites mentioned in Turn 2.


In [39]:
print("\n" + "=" * 80)
print("📍 TURN 3: Another Follow-up")
print("=" * 80)

demo_query_3 = "Do I meet those prerequisites?"
print(f"\n👤 User: {demo_query_3}")
print("   Note: 'those prerequisites' refers to prerequisites from Turn 2")

demo_response_3 = await memory_enhanced_rag_query(demo_query_3, sarah, demo_session_id)

print(f"\n🤖 Agent: {demo_response_3}")
print("\n✅ Agent resolved 'those prerequisites' and checked student's transcript!")

print("\n" + "=" * 80)
print("✅ DEMO COMPLETE: Memory-enhanced RAG enables natural conversations!")
print("=" * 80)


📍 TURN 3: Another Follow-up

👤 User: Do I meet those prerequisites?
   Note: 'those prerequisites' refers to prerequisites from Turn 2
19:17:27 httpx INFO   HTTP Request: GET http://localhost:8088/v1/working-memory/complete_demo_6ac53750?user_id=sarah.chen&namespace=redis_university&model_name=gpt-4o "HTTP/1.1 200 OK"


19:17:28 httpx INFO   HTTP Request: POST http://localhost:8088/v1/long-term-memory/search?optimize_query=false "HTTP/1.1 200 OK"


19:17:28 redis_context_course.hierarchical_manager INFO   Hierarchical search: 'Do I meet those prerequisites?' (summaries=3, details=2)


19:17:28 httpx INFO   HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


19:17:28 redis_context_course.hierarchical_manager INFO   Found 0 course summaries for query: Do I meet those prerequisites?


19:17:28 redis_context_course.hierarchical_manager INFO   Fetched 0 course details


19:17:28 redis_context_course.hierarchical_manager INFO   Hierarchical search complete: 0 summaries, 0 details


19:17:30 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


19:17:30 httpx INFO   HTTP Request: PUT http://localhost:8088/v1/working-memory/complete_demo_6ac53750?user_id=sarah.chen&model_name=gpt-4o "HTTP/1.1 200 OK"



🤖 Agent: Hi Sarah! Based on your completed courses and current enrollment, you meet the prerequisites for the courses I previously mentioned:

1. **Data Structures and Algorithms (CS301)**
   - Prerequisite: Completion of CS201 (which you've completed)

2. **Introduction to Data Science (DS101)**
   - Prerequisite: None specified, so you're all set to enroll.

Both courses align well with your interests in algorithms and data science, and they are offered online at an intermediate level, which matches your preferences. If you have any more questions or need further assistance, feel free to ask!

✅ Agent resolved 'those prerequisites' and checked student's transcript!

✅ DEMO COMPLETE: Memory-enhanced RAG enables natural conversations!


### 🎯 What Just Happened?

**Turn 1:** "I'm interested in machine learning courses"
- System uses hierarchical search (summaries → details)
- Finds ML-related courses
- Responds with recommendations
- **Saves conversation to working memory**

**Turn 2:** "What are the prerequisites for **the first one**?"
- System loads working memory (Turn 1)
- Resolves "the first one" → first course mentioned in Turn 1
- Responds with prerequisites
- **Saves updated conversation**

**Turn 3:** "Do I meet **those prerequisites**?"
- System loads working memory (Turns 1-2)
- Resolves "those prerequisites" → prerequisites from Turn 2
- Checks student's completed courses from profile
- Responds with personalized assessment

**Key Insight:** Memory transforms stateless RAG into stateful, personalized, context-aware conversations!


---

## 📊 Before vs. After Comparison

Let's visualize the difference between stateless and memory-enhanced RAG.

### **Stateless RAG (Module 2):**

```
Query 1: "I'm interested in ML courses"
  → ✅ Works (searches and returns courses)

Query 2: "What are the prerequisites for the first one?"
  → ❌ Fails (no conversation history)
  → Agent: "Which course are you referring to?"
```

**Problems:**
- ❌ No conversation continuity
- ❌ Can't resolve references
- ❌ Each query is independent
- ❌ Poor user experience

### **Memory-Enhanced RAG (This Module):**

```
Query 1: "I'm interested in ML courses"
  → ✅ Works (searches and returns courses)
  → Saves to working memory

Query 2: "What are the prerequisites for the first one?"
  → ✅ Works (loads conversation history)
  → Resolves "the first one" → first course from Query 1
  → Responds with prerequisites
  → Saves updated conversation

Query 3: "Do I meet those prerequisites?"
  → ✅ Works (loads conversation history)
  → Resolves "those prerequisites" → prerequisites from Query 2
  → Checks student transcript
  → Responds with personalized answer
```

**Benefits:**
- ✅ Conversation continuity
- ✅ Reference resolution
- ✅ Personalization
- ✅ Natural user experience

---

## 🎓 Key Takeaways

### **1. Memory Transforms RAG**

**Without Memory (Module 2):**
- Stateless queries
- No conversation continuity
- Limited to 3 context types (System, User, Retrieved)

**With Memory (This Module):**
- Stateful conversations
- Reference resolution
- All 4 context types (System, User, Conversation, Retrieved)

### **2. Two Types of Memory Work Together**

**Working Memory:**
- Session-scoped conversation history
- Enables reference resolution
- Persists within the session (like ChatGPT conversations)

**Long-term Memory:**
- User-scoped persistent facts
- Enables personalization
- Persists indefinitely

### **3. Hierarchical Retrieval + Memory**

**What We Built:**
- Combined hierarchical RAG (summaries → details) with memory
- Progressive disclosure pattern from Module 2
- Memory-enhanced context assembly
- All four context types working together

**Why This Matters:**
- Efficient token usage (progressive disclosure)
- Natural conversations (memory)
- Personalization (long-term memory)
- Foundation for agentic workflows (Module 5)

### **4. All Four Context Types**

| Context Type | Source | Purpose |
|--------------|--------|---------|
| **System Context** | Static prompt | Role, instructions, guidelines |
| **User Context** | Profile + long-term memories | Personalization |
| **Conversation Context** | Working memory | Reference resolution |
| **Retrieved Context** | Hierarchical RAG | Relevant information |

**Together:** Natural, stateful, personalized conversations

**💡 Research Insight (From Module 1):** Context Rot research demonstrates that context structure and organization affect LLM attention. Memory systems that selectively retrieve and organize context outperform systems that dump all available information. This validates our approach: quality over quantity, semantic similarity, and selective retrieval.

---

## 🏋️ Practice Exercises

### **Exercise 1: Cross-Session Personalization**

Modify the `memory_enhanced_rag_query` function to:
1. Store user preferences in long-term memory when mentioned
2. Use those preferences in future sessions
3. Test with two different sessions for the same student

**Hint:** Look for phrases like "I prefer...", "I like...", "I want..." and store them as semantic memories.

### **Exercise 2: Memory-Aware Filtering**

Enhance the hierarchical search to use long-term memories as filters:
1. Search long-term memory for preferences (format, difficulty, schedule)
2. Apply those preferences as filters to `hierarchical_manager.hierarchical_search()`
3. Compare results with and without memory-aware filtering

**Hint:** Use the `filters` parameter in the search methods.

### **Exercise 3: Conversation Summarization**

Implement a function that summarizes long conversations:
1. When working memory exceeds 10 messages, summarize the conversation
2. Store the summary in long-term memory
3. Clear old messages from working memory (keep only recent 4)
4. Test that reference resolution still works with summarized history

**Hint:** Use the LLM to generate summaries, then store as semantic memories.

### **Exercise 4: Multi-User Memory Management**

Create a simple CLI that:
1. Supports multiple students (different user IDs)
2. Maintains separate working memory per session
3. Maintains separate long-term memory per user
4. Demonstrates cross-session continuity for each user

**Hint:** Use different `session_id` and `user_id` for each student.

### **Exercise 5: Memory Search Quality**

Experiment with long-term memory search:
1. Store 20+ diverse memories for a student
2. Try different search queries
3. Analyze which memories are retrieved
4. Adjust memory text to improve search relevance

**Hint:** More specific memory text leads to better semantic search results.

---

## 📝 Summary

### **What You Learned:**

1. **The Grounding Problem** - Why agents need memory to resolve references
2. **Working Memory** - Session-scoped conversation history for continuity
3. **Long-term Memory** - Cross-session persistent knowledge for personalization
4. **Memory Integration** - Combining memory with Module 2's hierarchical RAG system
5. **Complete Context Engineering** - All four context types working together
6. **Production Architecture** - Using Agent Memory Server for scalable memory

### **What You Built:**

- ✅ Working memory demo (multi-turn conversations)
- ✅ Long-term memory demo (persistent knowledge)
- ✅ Complete memory-enhanced RAG system with hierarchical retrieval
- ✅ Integration of all four context types

### **Key Functions:**

- `memory_enhanced_rag_query()` - Complete memory + hierarchical RAG pipeline
- Working memory operations - Load, save, update conversation history
- Long-term memory operations - Store, search, filter persistent facts

### **Architecture Pattern:**

```
User Query
    ↓
Load Working Memory (conversation history)
    ↓
Search Long-term Memory (user facts)
    ↓
Hierarchical RAG Search (summaries → details)
    ↓
Assemble Context (System + User + Conversation + Retrieved)
    ↓
Generate Response
    ↓
Save Working Memory (updated conversation)
```

### **From Module 2 to Module 4:**

**Module 2 (Stateless RAG):**
- ❌ No conversation history
- ❌ Each query independent
- ❌ Can't resolve references
- ✅ Retrieves relevant documents
- ✅ Progressive disclosure (hierarchical)

**Module 4 (Memory-Enhanced RAG):**
- ✅ Conversation history (working memory)
- ✅ Multi-turn conversations
- ✅ Reference resolution
- ✅ Persistent user knowledge (long-term memory)
- ✅ Personalization across sessions
- ✅ Progressive disclosure (hierarchical)

### **Next Steps:**

**Module 5** will add **tools** and **agentic workflows** using **LangGraph**, completing your journey from context engineering fundamentals to production-ready AI agents.

---

## 🎉 Congratulations!

You've successfully built a **memory-enhanced RAG system** that:
- Remembers conversations (working memory)
- Accumulates knowledge (long-term memory)
- Resolves references naturally
- Personalizes responses
- Uses progressive disclosure (hierarchical retrieval)
- Integrates all four context types

**You're now ready for Module 5: Building Agents!** 🚀

---

## 📚 Additional Resources

- [Agent Memory Server Documentation](https://github.com/redis/agent-memory-server) - Production-ready memory management
- [Agent Memory Client](https://pypi.org/project/agent-memory-client/) - Python client for Agent Memory Server
- [RedisVL Documentation](https://redisvl.com/) - Redis Vector Library
- [LangChain Guide](https://python.langchain.com/docs/modules/memory/) - LangChain memory patterns
- [LangGraph Tutorials](https://langchain-ai.github.io/langgraph/tutorials/) - Building agents with LangGraph

---

![Redis](https://redis.io/wp-content/uploads/2024/04/Logotype.svg?auto=webp&quality=85,75&width=120)

**Redis University - Context Engineering Workshop**
